https://gist.github.com/ihsgnef/f13c35cd46624c8f458a4d23589ac768

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import configurations
import configurationsMulticlass
import funPytorch as fun
import pickle

In [2]:
def colorize(words, color_array, diverging=False, color=0):
    # words is a list of words
    # color_array is an array of numbers between 0 and 1 of length equal to words
    
    if diverging:
        template = 'RdBu'
        color_array = color_array / 2. + 0.5
    else:
        templates = ['Greens', 'Blues', 'Reds', 'Oranges', 'Purples', 'Greys']
        template = templates[color%len(templates)]
    cmap = matplotlib.cm.get_cmap(template)
    template = '<span class="barcode"; style="color: black; background-color: {}">{}</span>'
    colored_string = ''
    for word, color in zip(words, color_array):
        color = matplotlib.colors.rgb2hex(cmap(color)[:3])
        colored_string += template.format(color, '&nbsp' + word + '&nbsp')
    display(HTML(colored_string))
    return colored_string    

In [3]:
words = 'The quick brown fox jumps over the lazy dog'.split()
color_array = np.linspace(0,1,len(words))#np.random.rand(len(words))
s = colorize(words, color_array)
s = colorize(words, color_array, diverging = True)

In [4]:
color_array = np.linspace(-1,1,len(words))#np.random.rand(len(words))
s = colorize(words, color_array, diverging=True)
s = colorize(words, color_array)

In [5]:
device = "cuda:0"

#conf = configurations.config50
#conf = configurations.config50c
#conf = configurations.config51
#conf = configurationsMulticlass.config28
#conf = configurationsMulticlass.config30v

#conf = configurationsMulticlass.configTImulti1best
conf = configurationsMulticlass.configTSmulti1best

In [6]:
X, y, train, valid, test, allText = fun.processData(conf, getText=True)
model,optim = fun.loadModel(conf, device)

         processed line 85170/85170           
         reprocessed line 85170/85170           


In [7]:
dataset = valid

In [8]:
yp, att = fun.getAttention(conf, model, X, y, dataset)

In [9]:
if np.min(att) < 0.:
    att = att/2 + 0.5

In [10]:
yt = y[dataset]
Xt = X[dataset]
text = [allText[c] for c in dataset]

In [11]:
att.shape

(17034, 100)

In [12]:
if len(att.shape) == 2:
    att = att.reshape(att.shape[0], att.shape[1], 1)

### Visualize only phrase attention

In [13]:
binary = False
colonRetto = False
hiddenEqualClasses = False

In [14]:
if not binary:
    _,values,_,_,_ = fun.processCorpusAndSplit(conf)
    icdo = pickle.load(open("icdo-3-dict.p", 'rb'))
    icdoEn = pickle.load(open("icdo-3-dict-en.p", 'rb'))

In [15]:
[(i,c,icdo[c]) for i,c in enumerate(icdo)]

[(0, 0, 'LABBRO'),
 (1, 1, 'BASE DELLA LINGUA'),
 (2, 2, 'ALTRE PARTI E PARTI NON SPECIFICATE DELLA LINGUA'),
 (3, 3, 'GENGIVA'),
 (4, 4, 'PAVIMENTO ORALE'),
 (5, 5, 'PALATO'),
 (6, 6, 'ALTRE PARTI E PARTI NON SPECIFICATE DELLA BOCCA'),
 (7, 7, 'GHIANDOLA PAROTIDE'),
 (8, 8, 'ALTRE GHIANDOLE SALIVARI MAGGIORI E NON SPECIFICATE'),
 (9, 9, 'TONSILLA'),
 (10, 10, 'OROFARINGE'),
 (11, 11, 'RINOFARINGE'),
 (12, 12, 'SENO PIRIFORME'),
 (13, 13, 'IPOFARINGE'),
 (14,
  14,
  'ALTRE E MAL DEFINITE SEDI DEL LABBRO, DEL CAVO ORALE E DELLA FARINGE'),
 (15, 15, 'ESOFAGO'),
 (16, 16, 'STOMACO'),
 (17, 17, 'INTESTINO TENUE'),
 (18, 18, 'COLON'),
 (19, 19, 'GIUNZIONE RETTOSIGMOIDEA'),
 (20, 20, 'RETTO'),
 (21, 21, 'ANO E CANALE ANALE'),
 (22, 22, 'FEGATO E DOTTI BILIARI INTRAEPATICI'),
 (23, 23, 'COLECISTI'),
 (24, 24, 'ALTRE PARTI E PARTI NON SPECIFICATE DELLE VIE BILIARI'),
 (25, 25, 'PANCREAS'),
 (26, 26, 'ALTRI E MAL DEFINITI ORGANI DIGESTIVI'),
 (27, 30, 'CAVITÀ NASALE E ORECCHIO MEDIO'),
 (28, 3

In [16]:
inspectDoc = 923

In [17]:
np.where(np.max(att[inspectDoc],0)>0.1)

(array([0]),)

In [18]:
np.max(att[inspectDoc],0)[np.where(np.max(att[inspectDoc],0)>0.1)]

array([0.47194824], dtype=float32)

In [19]:
np.max(att[inspectDoc],0)[31]==np.max(att[inspectDoc],0)[67]

IndexError: index 31 is out of bounds for axis 0 with size 1

In [20]:
visRange = (0,len(yt))#(50,100)
visLimit = 50      #int or None
only=None#[values.index(32), values.index(33), values.index(34)]          #None, list
printWhat = 'all'  #all, max or maxed
attThreshold = 0.1  #None or float
attColors = ['red', 'green', 'blue', 'black']

In [21]:
printed = 0
for currDoc in range(visRange[0], visRange[1]):
    if binary and (not only is None) and (not yt[currDoc][0] in only):
        continue
    if (not binary) and (not only is None) and (not yt[currDoc] in only):
        continue
    print("======== doc: ", currDoc)
    if binary:
        currYt = yt[currDoc][0]
        currYp = yp[currDoc][0]
        print("y: {}; yp: {}".format(currYt, currYp))
    else:
        currYt = yt[currDoc]
        currYp = yp[currDoc]
        print("y: {} ({}-{}); yp: {} ({}-{})".format(currYt, values[currYt], icdo[values[currYt]], np.argmax(currYp), values[np.argmax(currYp)], icdo[values[np.argmax(currYp)]]))
        print(" - ".join(["{}:{:.1f}".format(i,p) for i,p in enumerate(currYp)]))
    if colonRetto:
        if currYt == 1.:
            print("Retto")
        elif currYt == 0.:
            print("Colon")
    print("---")
    for currAttNum in range(att[currDoc].shape[1]):
        if binary or not hiddenEqualClasses:
            print(currAttNum)
        else:
            print("{} ({}-{})".format(currAttNum, values[currAttNum], icdo[values[currAttNum]]))
        currAtt = att[currDoc][:,currAttNum]
        if printWhat == 'all':
            s = colorize(text[currDoc], currAtt, diverging=False, color=currAttNum)
        else:
            maxAtt = np.zeros_like(currAtt)
            if printWhat == 'maxed':
                maxAtt[np.argmax(currAtt)] = 1
            if printWhat == 'max':
                maxAtt[np.argmax(currAtt)] = currAtt[np.argmax(currAtt)]
            s = colorize(text[currDoc], maxAtt, diverging=False, color=currAttNum)
            
    printed += 1
    if (not visLimit is None) and (printed >= visLimit):
        break

======== doc:  0
y: 37 (44-CUTE); yp: 37 (44-CUTE)
0:-2.9 - 1:-8.1 - 2:-5.4 - 3:-7.6 - 4:-12.3 - 5:-7.5 - 6:-2.7 - 7:-3.9 - 8:-8.6 - 9:-7.9 - 10:-9.4 - 11:-6.3 - 12:-24.1 - 13:-12.1 - 14:-9.3 - 15:-8.6 - 16:-6.0 - 17:-8.2 - 18:-0.3 - 19:-7.4 - 20:-0.3 - 21:-0.4 - 22:-7.4 - 23:-9.8 - 24:-13.8 - 25:-9.7 - 26:-6.2 - 27:-2.2 - 28:-8.5 - 29:-4.7 - 30:-20.0 - 31:-5.3 - 32:-15.8 - 33:-9.7 - 34:-13.9 - 35:-5.0 - 36:-2.4 - 37:8.3 - 38:-9.1 - 39:-11.6 - 40:-0.3 - 41:-2.4 - 42:-0.9 - 43:-8.4 - 44:-6.2 - 45:-5.1 - 46:-10.3 - 47:-13.6 - 48:-13.7 - 49:-2.2 - 50:-4.0 - 51:-14.5 - 52:-4.6 - 53:-6.5 - 54:-22.0 - 55:-19.4 - 56:-3.3 - 57:-9.5 - 58:-3.7 - 59:-8.7 - 60:-6.4 - 61:-11.7 - 62:-9.5 - 63:-22.4 - 64:-7.0 - 65:-0.4 - 66:-3.8 - 67:-0.6
---
0


======== doc:  1
y: 37 (44-CUTE); yp: 37 (44-CUTE)
0:-1.0 - 1:-8.5 - 2:-5.6 - 3:-9.0 - 4:-13.6 - 5:-6.2 - 6:-1.9 - 7:-3.0 - 8:-10.5 - 9:-7.4 - 10:-11.0 - 11:-7.9 - 12:-23.3 - 13:-12.9 - 14:-9.2 - 15:-11.0 - 16:-10.1 - 17:-13.0 - 18:-0.3 - 19:-8.9 - 20:-0.3 - 21:-1.4 - 22:-8.6 - 23:-12.1 - 24:-17.6 - 25:-12.1 - 26:-8.9 - 27:-3.4 - 28:-10.8 - 29:-2.5 - 30:-19.2 - 31:-3.4 - 32:-16.9 - 33:-10.1 - 34:-19.5 - 35:-8.4 - 36:-2.8 - 37:9.9 - 38:-11.7 - 39:-16.6 - 40:-2.4 - 41:-1.4 - 42:0.9 - 43:-9.6 - 44:-4.4 - 45:-6.3 - 46:-10.6 - 47:-16.0 - 48:-16.4 - 49:-1.1 - 50:-3.4 - 51:-15.7 - 52:-6.6 - 53:-7.5 - 54:-25.2 - 55:-21.7 - 56:-4.3 - 57:-10.1 - 58:-5.0 - 59:-9.1 - 60:-7.3 - 61:-13.0 - 62:-9.4 - 63:-27.9 - 64:-8.8 - 65:-1.8 - 66:-4.9 - 67:-1.8
---
0


======== doc:  2
y: 20 (20-RETTO); yp: 21 (21-ANO E CANALE ANALE)
0:-11.0 - 1:-11.2 - 2:-9.1 - 3:-11.7 - 4:-16.1 - 5:-7.5 - 6:-5.4 - 7:-10.4 - 8:-13.6 - 9:-11.2 - 10:-10.3 - 11:-9.9 - 12:-30.0 - 13:-9.4 - 14:-8.7 - 15:-6.4 - 16:-3.3 - 17:-8.3 - 18:2.4 - 19:-1.8 - 20:3.3 - 21:4.6 - 22:-11.7 - 23:-8.1 - 24:-11.8 - 25:-10.7 - 26:-5.0 - 27:-2.1 - 28:-4.3 - 29:-7.5 - 30:-24.8 - 31:-6.7 - 32:-22.8 - 33:-12.2 - 34:-10.9 - 35:-9.3 - 36:-2.8 - 37:0.7 - 38:-18.1 - 39:-9.6 - 40:-2.9 - 41:-5.3 - 42:-9.5 - 43:-5.1 - 44:-2.7 - 45:0.1 - 46:-5.9 - 47:-7.7 - 48:-11.6 - 49:-9.8 - 50:-7.7 - 51:-17.3 - 52:-9.8 - 53:-10.2 - 54:-26.9 - 55:-16.8 - 56:-3.6 - 57:-10.6 - 58:-9.1 - 59:-5.1 - 60:-4.6 - 61:-5.8 - 62:-14.2 - 63:-18.0 - 64:-6.0 - 65:-4.0 - 66:-7.9 - 67:-4.8
---
0


======== doc:  3
y: 37 (44-CUTE); yp: 37 (44-CUTE)
0:-3.1 - 1:-14.0 - 2:-11.7 - 3:-11.3 - 4:-17.0 - 5:-9.3 - 6:-3.5 - 7:-5.1 - 8:-13.1 - 9:-8.7 - 10:-14.5 - 11:-9.7 - 12:-29.4 - 13:-18.1 - 14:-12.4 - 15:-15.5 - 16:-11.7 - 17:-15.3 - 18:-0.4 - 19:-11.3 - 20:-3.2 - 21:-1.6 - 22:-9.0 - 23:-11.8 - 24:-18.9 - 25:-12.5 - 26:-10.7 - 27:-3.4 - 28:-13.1 - 29:-4.0 - 30:-24.5 - 31:-3.1 - 32:-17.2 - 33:-10.0 - 34:-20.3 - 35:-8.2 - 36:-1.3 - 37:11.3 - 38:-9.2 - 39:-17.0 - 40:-2.3 - 41:-0.9 - 42:-0.6 - 43:-9.7 - 44:-4.8 - 45:-5.2 - 46:-10.5 - 47:-15.7 - 48:-18.6 - 49:-3.0 - 50:-3.8 - 51:-15.6 - 52:-6.0 - 53:-11.2 - 54:-32.9 - 55:-28.1 - 56:-7.1 - 57:-13.6 - 58:-2.6 - 59:-7.8 - 60:-7.0 - 61:-12.5 - 62:-10.8 - 63:-27.2 - 64:-5.9 - 65:-0.6 - 66:-3.5 - 67:-0.8
---
0


======== doc:  4
y: 20 (20-RETTO); yp: 18 (18-COLON)
0:-16.1 - 1:-18.6 - 2:-13.0 - 3:-16.4 - 4:-23.8 - 5:-10.6 - 6:-11.7 - 7:-11.7 - 8:-17.2 - 9:-18.8 - 10:-14.6 - 11:-11.7 - 12:-31.3 - 13:-11.7 - 14:-11.7 - 15:-7.0 - 16:-2.2 - 17:-1.8 - 18:6.5 - 19:2.9 - 20:6.1 - 21:1.3 - 22:-5.1 - 23:-6.3 - 24:-8.3 - 25:-5.6 - 26:0.0 - 27:-5.9 - 28:-6.2 - 29:-8.9 - 30:-21.7 - 31:-4.7 - 32:-21.4 - 33:-10.7 - 34:-11.0 - 35:-13.0 - 36:-6.5 - 37:-2.6 - 38:-19.2 - 39:-6.9 - 40:-3.4 - 41:-5.0 - 42:-13.4 - 43:-7.0 - 44:-5.8 - 45:-0.8 - 46:-5.2 - 47:-3.1 - 48:-11.9 - 49:-15.1 - 50:-10.3 - 51:-18.4 - 52:-16.2 - 53:-6.7 - 54:-24.2 - 55:-11.0 - 56:-4.0 - 57:-12.7 - 58:-12.7 - 59:-9.9 - 60:-4.7 - 61:-5.6 - 62:-9.1 - 63:-14.5 - 64:-4.2 - 65:-7.9 - 66:-14.7 - 67:-3.4
---
0


...

======== doc:  45
y: 18 (18-COLON); yp: 18 (18-COLON)
0:-12.0 - 1:-15.6 - 2:-9.3 - 3:-10.0 - 4:-16.7 - 5:-10.4 - 6:-7.0 - 7:-5.2 - 8:-9.4 - 9:-16.5 - 10:-12.2 - 11:-9.3 - 12:-27.0 - 13:-8.1 - 14:-8.2 - 15:-7.3 - 16:-1.7 - 17:0.5 - 18:7.1 - 19:1.8 - 20:4.5 - 21:-1.1 - 22:-5.5 - 23:-6.5 - 24:-7.9 - 25:-4.5 - 26:0.6 - 27:-5.9 - 28:-7.1 - 29:-6.7 - 30:-21.7 - 31:-4.5 - 32:-20.0 - 33:-8.8 - 34:-12.0 - 35:-9.6 - 36:-5.1 - 37:-0.6 - 38:-14.5 - 39:-4.5 - 40:-1.4 - 41:-4.4 - 42:-11.3 - 43:-7.9 - 44:-5.8 - 45:-2.0 - 46:-7.6 - 47:-2.6 - 48:-11.8 - 49:-11.1 - 50:-5.8 - 51:-14.0 - 52:-13.5 - 53:-1.2 - 54:-15.3 - 55:-7.2 - 56:-1.1 - 57:-9.3 - 58:-12.1 - 59:-11.1 - 60:-3.7 - 61:-7.3 - 62:-6.2 - 63:-14.4 - 64:-3.8 - 65:-4.0 - 66:-10.1 - 67:-2.7
---
0


======== doc:  46
y: 50 (61-PROSTATA); yp: 50 (61-PROSTATA)
0:-23.1 - 1:-13.1 - 2:-14.0 - 3:-27.2 - 4:-21.8 - 5:-11.6 - 6:-15.3 - 7:-11.9 - 8:-14.4 - 9:-10.1 - 10:-13.9 - 11:-19.3 - 12:-17.7 - 13:-15.9 - 14:-12.8 - 15:-16.9 - 16:-10.4 - 17:-17.3 - 18:-2.8 - 19:-7.8 - 20:-10.2 - 21:-16.9 - 22:-7.8 - 23:-1.2 - 24:-8.8 - 25:-9.2 - 26:-12.1 - 27:-15.4 - 28:-21.4 - 29:-8.8 - 30:-11.1 - 31:-1.6 - 32:-19.3 - 33:-14.4 - 34:-37.0 - 35:-31.6 - 36:-4.1 - 37:-1.6 - 38:-26.0 - 39:-14.3 - 40:-10.1 - 41:-10.5 - 42:-15.9 - 43:-20.8 - 44:-8.7 - 45:-3.0 - 46:-12.4 - 47:-10.8 - 48:-15.0 - 49:-17.1 - 50:8.6 - 51:-12.8 - 52:-28.7 - 53:-1.2 - 54:-19.2 - 55:-13.6 - 56:-1.6 - 57:-3.6 - 58:-16.8 - 59:-5.9 - 60:-12.7 - 61:-18.5 - 62:-4.6 - 63:-13.9 - 64:-7.5 - 65:-12.7 - 66:-7.6 - 67:-5.2
---
0


======== doc:  47
y: 37 (44-CUTE); yp: 37 (44-CUTE)
0:-16.0 - 1:-17.3 - 2:-15.3 - 3:-13.2 - 4:-24.0 - 5:-6.6 - 6:-6.5 - 7:-11.2 - 8:-14.2 - 9:-14.7 - 10:-14.8 - 11:-8.5 - 12:-39.7 - 13:-18.5 - 14:-14.8 - 15:-16.0 - 16:-9.7 - 17:-8.8 - 18:0.5 - 19:-8.9 - 20:-2.4 - 21:-0.9 - 22:-11.7 - 23:-14.3 - 24:-22.4 - 25:-15.6 - 26:-7.4 - 27:-5.1 - 28:-12.0 - 29:-8.1 - 30:-30.6 - 31:-9.1 - 32:-14.1 - 33:-8.2 - 34:-14.3 - 35:-4.4 - 36:-3.2 - 37:5.2 - 38:-8.7 - 39:-8.1 - 40:4.5 - 41:-3.8 - 42:-7.5 - 43:-9.1 - 44:-10.8 - 45:-2.9 - 46:-11.0 - 47:-13.1 - 48:-14.4 - 49:-10.6 - 50:-12.8 - 51:-17.2 - 52:-5.7 - 53:-10.4 - 54:-31.4 - 55:-26.1 - 56:-4.2 - 57:-13.5 - 58:-7.2 - 59:-4.7 - 60:-3.3 - 61:-4.3 - 62:-10.3 - 63:-17.7 - 64:-3.4 - 65:-2.2 - 66:-5.2 - 67:-3.4
---
0


======== doc:  48
y: 37 (44-CUTE); yp: 37 (44-CUTE)
0:-2.4 - 1:-10.1 - 2:-7.6 - 3:-8.5 - 4:-12.6 - 5:-7.0 - 6:-2.1 - 7:-2.7 - 8:-9.2 - 9:-7.5 - 10:-11.1 - 11:-8.2 - 12:-23.8 - 13:-13.3 - 14:-9.5 - 15:-13.2 - 16:-10.0 - 17:-13.0 - 18:-0.2 - 19:-10.2 - 20:-1.8 - 21:-2.0 - 22:-8.4 - 23:-10.9 - 24:-16.6 - 25:-11.6 - 26:-9.6 - 27:-3.0 - 28:-10.2 - 29:-3.4 - 30:-19.5 - 31:-3.6 - 32:-15.6 - 33:-9.4 - 34:-18.5 - 35:-7.4 - 36:-2.1 - 37:9.9 - 38:-8.8 - 39:-14.8 - 40:-1.8 - 41:-1.3 - 42:-0.9 - 43:-8.9 - 44:-5.8 - 45:-6.1 - 46:-11.3 - 47:-15.7 - 48:-16.2 - 49:-2.2 - 50:-3.3 - 51:-15.0 - 52:-6.7 - 53:-8.5 - 54:-25.4 - 55:-21.7 - 56:-4.1 - 57:-10.4 - 58:-2.6 - 59:-7.3 - 60:-4.9 - 61:-10.3 - 62:-8.2 - 63:-24.1 - 64:-5.3 - 65:-1.0 - 66:-3.9 - 67:-1.3
---
0


======== doc:  49
y: 50 (61-PROSTATA); yp: 50 (61-PROSTATA)
0:-18.8 - 1:-11.7 - 2:-12.3 - 3:-23.8 - 4:-19.6 - 5:-10.0 - 6:-12.8 - 7:-10.8 - 8:-13.3 - 9:-8.2 - 10:-12.7 - 11:-17.6 - 12:-12.8 - 13:-13.2 - 14:-11.1 - 15:-14.6 - 16:-10.6 - 17:-16.7 - 18:-3.9 - 19:-8.3 - 20:-9.7 - 21:-15.1 - 22:-7.3 - 23:-1.3 - 24:-8.7 - 25:-9.3 - 26:-12.4 - 27:-12.9 - 28:-19.0 - 29:-5.9 - 30:-9.2 - 31:-0.7 - 32:-17.7 - 33:-13.2 - 34:-34.0 - 35:-28.8 - 36:-5.2 - 37:-0.7 - 38:-23.6 - 39:-14.7 - 40:-9.4 - 41:-10.4 - 42:-12.9 - 43:-18.8 - 44:-6.4 - 45:-2.7 - 46:-10.4 - 47:-10.2 - 48:-14.1 - 49:-14.2 - 50:8.0 - 51:-11.2 - 52:-25.2 - 53:-2.2 - 54:-19.0 - 55:-13.5 - 56:-2.2 - 57:-3.9 - 58:-14.5 - 59:-5.7 - 60:-12.9 - 61:-17.8 - 62:-4.7 - 63:-15.5 - 64:-7.8 - 65:-12.3 - 66:-6.5 - 67:-5.6
---
0


In [40]:
printed = 0
for currDoc in range(visRange[0], visRange[1]):
    if binary and (not only is None) and (not yt[currDoc][0] in only):
        continue
    if (not binary) and (not only is None) and (not yt[currDoc] in only):
        continue
    print("======== doc: ", currDoc)
    if binary:
        currYt = yt[currDoc][0]
        currYp = yp[currDoc][0]
        print("y: {}; yp: {}".format(currYt, currYp))
    else:
        currYt = yt[currDoc]
        currYp = yp[currDoc]
        print("y: {} ({}-{}); yp: {} ({}-{})".format(currYt, values[currYt], icdo[values[currYt]], np.argmax(currYp), values[np.argmax(currYp)], icdo[values[np.argmax(currYp)]]))
        #print(currYp)
    if colonRetto:
        if currYt == 1.:
            print("Retto")
        elif currYt == 0.:
            print("Colon")
   
    attList = []
    print("Relevant attentions:")
    for currAttNum in range(att[currDoc].shape[1]):
        if np.max(att[currDoc][:,currAttNum]) > attThreshold:
            print("{} ({}-{})".format(currAttNum, values[currAttNum], icdo[values[currAttNum]]))
            attList.append("{} ({})".format(values[currAttNum], icdoEn[values[currAttNum]]))
    print("---")
    print("{}".format(values[currYt]), end="")
    print("&", end="")
    #print("{}".format(values[np.argmax(currYp)]), end="")
    #print("&", end="")
    print("\\begin{minipage}{\\attTableIcdoWidth}", end="")
    for i in range(len(attList)):
        if i>0:
            print("\\\\\\\\", end="")
        for currWord in attList[i].split():
            for j in range(len(attList)):
                print("\\att{",end="")
            print(currWord, end="")
            currAttColor = 0
            for j in range(len(attList)):
                if j == i:
                    currAttColorStrong = 100
                else:
                    currAttColorStrong = 0
                print("}}{{{}}}{{{:.0f}}}".format(attColors[currAttColor], currAttColorStrong), end="")
                currAttColor += 1
                if currAttColor == len(attColors):
                    currAttColor = 0
            print(" ", end="")
    print("\\end{minipage}", end="")
    print("&", end="")
    print("\\begin{minipage}{\\attTableTextWidth}", end="")
    for indexWord, word in enumerate(text[currDoc]):
        word = word.replace("#", "\#")
        word = word.replace("%", "\%")
        #for currAttNum in range(28,33):
        for currAttNum in range(att[currDoc].shape[1]):
            if np.max(att[currDoc][:,currAttNum]) > attThreshold:
                print("\\att{",end="")
        print(word, end="")
        currAttColor = 0
        #for currAttNum in range(28,33):
        for currAttNum in range(att[currDoc].shape[1]):
            currAtt = att[currDoc][:,currAttNum]
            if np.max(currAtt) > attThreshold:
                if printWhat == 'all':
                    print("}}{{{}}}{{{:.0f}}}".format(attColors[currAttColor], currAtt[indexWord]*100), end="")
                else:
                    maxAtt = np.zeros_like(currAtt)
                    if printWhat == 'maxed':
                        maxAtt[np.argmax(currAtt)] = 1
                    if printWhat == 'max':
                        maxAtt[np.argmax(currAtt)] = currAtt[np.argmax(currAtt)]
                    print("}}{{{}}}{{{:.0f}}}".format(attColors[currAttColor], maxAtt[indexWord]*100), end="")
                currAttColor += 1
                if currAttColor == len(attColors):
                    currAttColor = 0
        print(" ", end="")
    print("\\end{minipage}", end="")
    print("\\\\")
    printed += 1
    if (not visLimit is None) and (printed >= visLimit):
        break

======== doc:  837
y: 31 (34-BRONCO E POLMONE); yp: 31 (34-BRONCO E POLMONE)
Relevant attentions:
31 (34-BRONCO E POLMONE)
---
34&\begin{minipage}{\attTableIcdoWidth}\att{34}{red}{100} \att{(BRONCHUS}{red}{100} \att{AND}{red}{100} \att{LUNG)}{red}{100} \end{minipage}&\begin{minipage}{\attTableTextWidth}\att{REPERTI}{red}{0} \att{MORFOLOGICI}{red}{0} \att{ED}{red}{0} \att{IMMUNOISTOCHIMICI}{red}{0} \att{(}{red}{0} \att{CK7}{red}{0} \att{+}{red}{0} \att{,}{red}{0} \att{CD56}{red}{43} \att{+}{red}{0} \att{,}{red}{0} \att{NSE}{red}{0} \att{+}{red}{0} \att{,}{red}{0} \att{CROMOGRANINA}{red}{0} \att{+}{red}{0} \att{,}{red}{0} \att{LCA}{red}{0} \att{-}{red}{0} \att{)}{red}{0} \att{COMPATIBILI}{red}{0} \att{CON}{red}{0} \att{CARCINOMA}{red}{0} \att{A}{red}{0} \att{PICCOLE}{red}{100} \att{CELLULE}{red}{99} \att{.}{red}{0} \att{A}{red}{0} \att{)}{red}{0} \att{BIOPSIA}{red}{0} \att{BRONCHIALE}{red}{100} \att{DEL}{red}{0} \att{POLMONE}{red}{100} \att{SINISTRO}{red}{100} \att{(}{red}{0} \att{PRELIE

34&\begin{minipage}{\attTableIcdoWidth}\att{\att{\att{\att{34}{red}{100}}{green}{0}}{blue}{0}}{black}{0} \att{\att{\att{\att{(BRONCHUS}{red}{100}}{green}{0}}{blue}{0}}{black}{0} \att{\att{\att{\att{AND}{red}{100}}{green}{0}}{blue}{0}}{black}{0} \att{\att{\att{\att{LUNG)}{red}{100}}{green}{0}}{blue}{0}}{black}{0} \\\\\att{\att{\att{\att{56}{red}{0}}{green}{100}}{blue}{0}}{black}{0} \att{\att{\att{\att{(OVARY)}{red}{0}}{green}{100}}{blue}{0}}{black}{0} \\\\\att{\att{\att{\att{67}{red}{0}}{green}{0}}{blue}{100}}{black}{0} \att{\att{\att{\att{(BLADDER)}{red}{0}}{green}{0}}{blue}{100}}{black}{0} \\\\\att{\att{\att{\att{80}{red}{0}}{green}{0}}{blue}{0}}{black}{100} \att{\att{\att{\att{(UNKNOWN}{red}{0}}{green}{0}}{blue}{0}}{black}{100} \att{\att{\att{\att{PRIMARY}{red}{0}}{green}{0}}{blue}{0}}{black}{100} \att{\att{\att{\att{SITE)}{red}{0}}{green}{0}}{blue}{0}}{black}{100} \end{minipage}&\begin{minipage}{\attTableTextWidth}\att{\att{\att{\att{VERSAMENTO}{red}{5}}{green}{0}}{blue}{0}}{black}{

32&\begin{minipage}{\attTableIcdoWidth}\att{32}{red}{100} \att{(LARYNX)}{red}{100} \end{minipage}&\begin{minipage}{\attTableTextWidth}\att{CARCINOMA}{red}{0} \att{A}{red}{0} \att{CELLULE}{red}{0} \att{SQUAMOSE}{red}{14} \att{LARINGEO}{red}{100} \att{A}{red}{0} \att{MEDIO}{red}{0} \att{GRADO}{red}{0} \att{DI}{red}{0} \att{DIFFERENZIAZIONE}{red}{0} \att{(}{red}{0} \att{G2}{red}{0} \att{)}{red}{0} \att{(}{red}{0} \att{2}{red}{0} \att{-}{red}{0} \att{5}{red}{0} \att{)}{red}{0} \att{.}{red}{0} \att{IMBIBIZIONE}{red}{0} \att{EDEMATOSA}{red}{0} \att{DEL}{red}{0} \att{CORION}{red}{0} \att{LARINGEO}{red}{100} \att{(}{red}{0} \att{1}{red}{0} \att{)}{red}{0} \att{.}{red}{0} \att{FRAMMENTI}{red}{0} \att{DI}{red}{0} \att{MUCOSA}{red}{0} \att{LARINGEA}{red}{100} \att{:}{red}{0} \att{1}{red}{0} \att{)}{red}{0} \att{FRAMMENTI}{red}{0} \att{(}{red}{0} \att{NR}{red}{0} \att{.}{red}{0} \att{3}{red}{0} \att{)}{red}{0} \att{DI}{red}{0} \att{PROLASSO}{red}{0} \att{VENTRICOLO}{red}{0} \att{SN}{red}{0} \att{;